<a href="https://colab.research.google.com/github/AkritiGhosh/KrayenAssessment/blob/main/GarbageClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Kaggle API Processing to download dataset

In [1]:
!pip install kaggle
# Create .kaggle folder in ../root directory
!mkdir ~/.kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# Upload kaggle.json => Personal API Token downloaded from kaggle
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"akritighosh","key":"6338b709da906e0e129e6ff19fe17260"}'}

In [3]:
# Copy kaggle.json file to created .kaggle folder
!cp kaggle.json ~/.kaggle/
# Change permissions for the file
!chmod 600 ~/.kaggle/kaggle.json

## Download dataset zip 

In [4]:
!kaggle datasets download asdasdasasdas/garbage-classification

 79% 65.0M/82.0M [00:03<00:00, 21.6MB/s]
100% 82.0M/82.0M [00:03<00:00, 24.1MB/s]


## Unzip the compressed dataset

In [5]:
!unzip garbage-classification.zip -d data

Streaming output truncated to the last 5000 lines.
  inflating: data/Garbage classification/Garbage classification/cardboard/cardboard152.jpg  
  inflating: data/Garbage classification/Garbage classification/cardboard/cardboard153.jpg  
  inflating: data/Garbage classification/Garbage classification/cardboard/cardboard154.jpg  
  inflating: data/Garbage classification/Garbage classification/cardboard/cardboard155.jpg  
  inflating: data/Garbage classification/Garbage classification/cardboard/cardboard156.jpg  
  inflating: data/Garbage classification/Garbage classification/cardboard/cardboard157.jpg  
  inflating: data/Garbage classification/Garbage classification/cardboard/cardboard158.jpg  
  inflating: data/Garbage classification/Garbage classification/cardboard/cardboard159.jpg  
  inflating: data/Garbage classification/Garbage classification/cardboard/cardboard16.jpg  
  inflating: data/Garbage classification/Garbage classification/cardboard/cardboard160.jpg  
  inflating: data/Ga

## Delete duplicate image folder

In [6]:
!rm -rf '/content/data/Garbage classification'

# Import necessary libraries

In [7]:
# Interacting with the operating system and perform file management tasks
import shutil
# To create, edit and process dataset from csv files
import pandas as pd
# For mathematical operations and multidimensional arrays
import numpy as np
import math
# For image augmentation
from tensorflow.keras.preprocessing.image import  load_img, img_to_array, ImageDataGenerator
# For creating model
import tensorflow as tf

# Import labelled data for train, validation and test dataset

In [8]:
classes = {1:'glass', 2:'paper',3:'cardboard',4:'plastic',5:'metal',6:'trash'}

In [9]:
df = pd.read_csv('/content/data/one-indexed-files.txt',   header=None, names=['File','Label'], sep=' ')
df_train = pd.read_csv('/content/data/one-indexed-files-notrash_train.txt',   header=None, names=['File','Label'], sep=' ')
df_val = pd.read_csv('/content/data/one-indexed-files-notrash_val.txt',   header=None, names=['File','Label'], sep=' ')
df_test = pd.read_csv('/content/data/one-indexed-files-notrash_test.txt',   header=None, names=['File','Label'], sep=' ')

## Train dataset

In [10]:
# Dataframe info
print(df_train.info())
# Get shape
print('No of rows', df_train.shape[0])
df_train

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1768 entries, 0 to 1767
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   File    1768 non-null   object
 1   Label   1768 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 27.8+ KB
None
No of rows 1768


,File,Label
0,cardboard202.jpg,3
1,paper472.jpg,2
2,paper522.jpg,2
3,glass189.jpg,1
4,glass325.jpg,1
...,...,...
1763,cardboard6.jpg,3
1764,glass283.jpg,1
1765,metal335.jpg,5
1766,plastic133.jpg,4


## Validation dataset

In [11]:
# Dataframe info
print(df_val.info())
# Get shape
print('No of rows', df_val.shape[0])
df_val

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 328 entries, 0 to 327
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   File    328 non-null    object
 1   Label   328 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 5.2+ KB
None
No of rows 328


,File,Label
0,cardboard114.jpg,3
1,plastic204.jpg,4
2,glass123.jpg,1
3,glass152.jpg,1
4,glass398.jpg,1
...,...,...
323,metal58.jpg,5
324,cardboard149.jpg,3
325,plastic5.jpg,4
326,glass23.jpg,1


## Test dataset

In [12]:
# Dataframe info
print(df_test.info())
# Get shape
print('No of rows', df_test.shape[0])
df_test

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 431 entries, 0 to 430
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   File    431 non-null    object
 1   Label   431 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 6.9+ KB
None
No of rows 431


,File,Label
0,paper70.jpg,2
1,paper380.jpg,2
2,cardboard31.jpg,3
3,glass12.jpg,1
4,paper169.jpg,2
...,...,...
426,metal389.jpg,5
427,paper303.jpg,2
428,paper405.jpg,2
429,paper465.jpg,2


# Create the dataset as per requirement


### Current format of image dataset : -
data / <br/>
. class 1/ <br/>
.... img1 <br/>
.... img2 <br/>
. class 2/ <br/>
. class 3/ <br/>
. class 4/ <br/>


### Format required - 
data / <br/>
.. train / <br/>
.... class 1/ <br/>
....... img1 <br/>
....... img2 <br/>
.... class 2/ <br/>
.... class 3/ <br/>
.... class 4/ <br/>
.. test / <br/>
.... class 1/ <br/>
....... img1 <br/>
....... img2 <br/>
.... class 2/ <br/>
.... class 3/ <br/>
.... class 4/ <br/>


## Train directory

In [13]:
!mkdir '/content/data/train'
!mkdir '/content/data/train/glass'
!mkdir '/content/data/train/paper'
!mkdir '/content/data/train/cardboard'
!mkdir '/content/data/train/plastic'
!mkdir '/content/data/train/metal'
!mkdir '/content/data/train/trash'

In [14]:
for i in range(df_train.shape[0]):
  try:
    folder = classes[int(df_train.iloc[i,1])]
    new_path = '/content/data/train/' + folder
    # old_path = '/content/data/garbage classification/Garbage classification/', folder , str(df_train.iloc[i,0]))
    old_path = '/content/data/garbage classification/Garbage classification/' + folder + '/' + str(df_train.iloc[i,0])
    shutil.move(old_path, new_path)
  except:
    print(i, str(df_train.iloc[i,0]))

## Validation directory

In [15]:
!mkdir '/content/data/val'
!mkdir '/content/data/val/glass'
!mkdir '/content/data/val/paper'
!mkdir '/content/data/val/cardboard'
!mkdir '/content/data/val/plastic'
!mkdir '/content/data/val/metal'
!mkdir '/content/data/val/trash'

In [16]:
for i in range(df_val.shape[0]):
  try:
    folder = classes[int(df_val.iloc[i,1])]
    new_path = '/content/data/val/' + folder
    # old_path = '/content/data/garbage classification/Garbage classification/', folder , str(df_val.iloc[i,0]))
    old_path = '/content/data/garbage classification/Garbage classification/' + folder + '/' + str(df_val.iloc[i,0])
    shutil.move(old_path, new_path)
  except:
    print(i, str(df_val.iloc[i,0]))

## Test directory

In [17]:
!mkdir '/content/data/test'
!mkdir '/content/data/test/glass'
!mkdir '/content/data/test/paper'
!mkdir '/content/data/test/cardboard'
!mkdir '/content/data/test/plastic'
!mkdir '/content/data/test/metal'
!mkdir '/content/data/test/trash'

In [18]:
for i in range(df_test.shape[0]):
  try:
    folder = classes[int(df_test.iloc[i,1])]
    new_path = '/content/data/test/' + folder
    # old_path = '/content/data/garbage classification/Garbage classification/', folder , str(df_test.iloc[i,0]))
    old_path = '/content/data/garbage classification/Garbage classification/' + folder + '/' + str(df_test.iloc[i,0])
    shutil.move(old_path, new_path)
  except:
    print(i, str(df_test.iloc[i,0]))

## Delete old directory

In [19]:
!rm -rf '/content/data/garbage classification'

# Data Augmentation

In [20]:
data_gen = ImageDataGenerator(
        rotation_range=45,
        # width_shift_range=0.2,
        # height_shift_range=0.2,
        horizontal_flip=True,
        vertical_flip=True,
        # brightness_range=[0.75,1.25],
        rescale=1./255,
        # zoom_range=0.25,
        # fill_mode='nearest'
        )

test_gen = ImageDataGenerator(rescale=1./255)

In [50]:
val_data_gen.class_indices

{'cardboard': 0, 'glass': 1, 'metal': 2, 'paper': 3, 'plastic': 4, 'trash': 5}

In [21]:
train_data_gen = data_gen.flow_from_directory(
                                        batch_size=128,                           # number of images in each batch
                                        directory='/content/data/train',          # the source of your images
                                        shuffle=True,                             # the images will be shuffle
                                        target_size=(128, 128),                   # size (height, width) of your images
                                        class_mode='categorical'                  # categorical because the dataset contains more than 2 labels
                                        )
val_data_gen = data_gen.flow_from_directory(
                                        batch_size=128,                           # number of images in each batch
                                        directory='/content/data/val',            # the source of your images
                                        shuffle=True,                             # the images will be shuffle
                                        target_size=(128, 128),                   # size (height, width) of your images
                                        class_mode='categorical'                  # categorical because the dataset contains more than 2 labels
                                        )
val_test_gen = test_gen.flow_from_directory(
                                        batch_size=128,                           # number of images in each batch
                                        directory='/content/data/val',            # the source of your images
                                        shuffle=True,                             # the images will be shuffle
                                        target_size=(128, 128),                   # size (height, width) of your images
                                        class_mode='categorical'                  # categorical because the dataset contains more than 2 labels
                                        )
test_data_gen = test_gen.flow_from_directory(
                                        batch_size=128,                           # number of images in each batch
                                        directory='/content/data/test',           # the source of your images
                                        shuffle=True,                             # the images will be shuffle
                                        target_size=(128, 128),                   # size (height, width) of your images
                                        class_mode='categorical'                  # categorical because the dataset contains more than 2 labels
                                        )

Found 1768 images belonging to 6 classes.
Found 328 images belonging to 6 classes.
Found 328 images belonging to 6 classes.
Found 431 images belonging to 6 classes.


# Model Training

* Validation - val_data_gen
* 1 conv layer - 16 with batch normalization and dropout 
* 2 32 dense layers with batch normalization

In [36]:
cnn = tf.keras.models.Sequential([
                                    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape =(128,128,3)),
                                    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
                                    tf.keras.layers.MaxPooling2D(2,2),
                                  tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
                                  tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
                                    tf.keras.layers.MaxPooling2D(2,2),
                                  tf.keras.layers.Conv2D(512, (3,3), activation='relu'),
                                  tf.keras.layers.Conv2D(512, (3,3), activation='relu'),
                                    tf.keras.layers.MaxPooling2D(2,2),
                                    # tf.keras.layers.BatchNormalization(),
                                    # tf.keras.layers.Dropout(0.5),

                                    tf.keras.layers.Flatten(),
                                    # tf.keras.layers.Dense(64, activation='relu'),
                                    tf.keras.layers.Dense(1024, activation='relu'),
                                    tf.keras.layers.Dropout(0.5),
                                    tf.keras.layers.Dense(1024, activation='relu'),
                                    tf.keras.layers.Dropout(0.2),
                                    tf.keras.layers.Dense(6, activation='softmax')
])
cnn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_10 (Conv2D)          (None, 126, 126, 64)      1792      
                                                                 
 conv2d_11 (Conv2D)          (None, 124, 124, 64)      36928     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 62, 62, 64)       0         
 2D)                                                             
                                                                 
 conv2d_12 (Conv2D)          (None, 60, 60, 128)       73856     
                                                                 
 conv2d_13 (Conv2D)          (None, 58, 58, 128)       147584    
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 29, 29, 128)      0         
 2D)                                                  

In [37]:
# decrt = 0.1/120
# cnn.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate = 0.1, decay = decrt), metrics=['accuracy'])
cnn.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate = 0.0001), metrics=['accuracy'])
es = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=10, verbose=1, restore_best_weights=True)

cnn.fit(
  train_data_gen,
  batch_size = 32, 
  callbacks=[es], 
  validation_data=val_test_gen, 
  epochs=120
)

Epoch 1/120
14/14 [==============================] - 17s 1s/step - loss: 1.7358 - accuracy: 0.2189 - val_loss: 1.6843 - val_accuracy: 0.2530
Epoch 2/120
14/14 [==============================] - 14s 957ms/step - loss: 1.6644 - accuracy: 0.2715 - val_loss: 1.5936 - val_accuracy: 0.3750
Epoch 3/120
14/14 [==============================] - 13s 947ms/step - loss: 1.5435 - accuracy: 0.3795 - val_loss: 1.4976 - val_accuracy: 0.4055
Epoch 4/120
14/14 [==============================] - 13s 944ms/step - loss: 1.4322 - accuracy: 0.4276 - val_loss: 1.3991 - val_accuracy: 0.4238
Epoch 5/120
14/14 [==============================] - 13s 933ms/step - loss: 1.3695 - accuracy: 0.4446 - val_loss: 1.3826 - val_accuracy: 0.4207
Epoch 6/120
14/14 [==============================] - 13s 951ms/step - loss: 1.3481 - accuracy: 0.4598 - val_loss: 1.3992 - val_accuracy: 0.4360
Epoch 7/120
14/14 [==============================] - 14s 951ms/step - loss: 1.2965 - accuracy: 0.4712 - val_loss: 1.3031 - val_accuracy: 0.

In [24]:
cnn.fit(
  train_data_gen,
  batch_size = 16,
  callbacks=[es], 
  validation_data=val_test_gen, 
  epochs=120
)

Epoch 1/120
14/14 [==============================] - 13s 928ms/step - loss: 0.3280 - accuracy: 0.8824 - val_loss: 0.6943 - val_accuracy: 0.7866
Epoch 2/120
14/14 [==============================] - 13s 932ms/step - loss: 0.3499 - accuracy: 0.8750 - val_loss: 1.0694 - val_accuracy: 0.6463
Epoch 3/120
14/14 [==============================] - 12s 876ms/step - loss: 0.3298 - accuracy: 0.8812 - val_loss: 0.8485 - val_accuracy: 0.7287
Epoch 4/120
14/14 [==============================] - 12s 880ms/step - loss: 0.3399 - accuracy: 0.8863 - val_loss: 0.8402 - val_accuracy: 0.7287
Epoch 5/120
14/14 [==============================] - 13s 895ms/step - loss: 0.3103 - accuracy: 0.8891 - val_loss: 0.6840 - val_accuracy: 0.7957
Epoch 6/120
14/14 [==============================] - 12s 873ms/step - loss: 0.3226 - accuracy: 0.8846 - val_loss: 0.9336 - val_accuracy: 0.7591
Epoch 7/120
14/14 [==============================] - 12s 870ms/step - loss: 0.2974 - accuracy: 0.8971 - val_loss: 0.7535 - val_accuracy:

In [25]:
cnn.fit(
  train_data_gen,
  batch_size = 64,
  callbacks=[es], 
  validation_data=val_test_gen, 
  epochs=120
)

Epoch 1/120
14/14 [==============================] - 13s 934ms/step - loss: 0.3058 - accuracy: 0.8835 - val_loss: 0.9975 - val_accuracy: 0.6982
Epoch 2/120
14/14 [==============================] - 13s 892ms/step - loss: 0.3123 - accuracy: 0.8835 - val_loss: 0.8918 - val_accuracy: 0.7500
Epoch 3/120
14/14 [==============================] - 12s 872ms/step - loss: 0.3139 - accuracy: 0.8931 - val_loss: 1.3337 - val_accuracy: 0.6463
Epoch 4/120
14/14 [==============================] - 12s 872ms/step - loss: 0.3463 - accuracy: 0.8857 - val_loss: 1.4913 - val_accuracy: 0.6159
Epoch 5/120
14/14 [==============================] - 12s 871ms/step - loss: 0.3101 - accuracy: 0.8971 - val_loss: 0.9463 - val_accuracy: 0.7195
Epoch 6/120
14/14 [==============================] - 12s 871ms/step - loss: 0.2936 - accuracy: 0.8993 - val_loss: 1.1017 - val_accuracy: 0.7165
Epoch 7/120
14/14 [==============================] - 13s 893ms/step - loss: 0.2546 - accuracy: 0.9129 - val_loss: 0.8676 - val_accuracy:

In [40]:
cnn.save('/content/70val77train.h5')

In [27]:
from keras.applications.vgg16 import VGG16
from keras.models import Model

In [28]:
conv_base = VGG16(include_top=False,
                     weights='imagenet', 
                     input_shape=(128,128,3))
for layer in conv_base.layers:
  layer.trainable = False
top_model = conv_base.output
top_model = tf.keras.layers.Flatten(name="flatten")(top_model)
top_model = tf.keras.layers.Dense(4096, activation='relu')(top_model)
top_model = tf.keras.layers.Dense(1072, activation='relu')(top_model)
top_model = tf.keras.layers.Dropout(0.2)(top_model)
output_layer = tf.keras.layers.Dense(6, activation='softmax')(top_model)

# Group the convolutional base and new fully-connected layers into a Model object.
model = Model(inputs=conv_base.input, outputs=output_layer)


In [29]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 128, 128, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 128, 128, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 64, 64, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 64, 64, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 64, 64, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 32, 32, 128)       0     

In [30]:
# Compiles the model for training.
model.compile(optimizer='adam', 
              loss='categorical_crossentropy',
              metrics=['accuracy'])
vgg_history = model.fit(train_data_gen, batch_size = 64, callbacks=[es], validation_data=val_test_gen, epochs=120)

Epoch 1/120
14/14 [==============================] - 21s 1s/step - loss: 4.3833 - accuracy: 0.2579 - val_loss: 1.4882 - val_accuracy: 0.3994
Epoch 2/120
14/14 [==============================] - 12s 849ms/step - loss: 1.2722 - accuracy: 0.4966 - val_loss: 1.1095 - val_accuracy: 0.5762
Epoch 3/120
14/14 [==============================] - 12s 865ms/step - loss: 1.0456 - accuracy: 0.5995 - val_loss: 0.9435 - val_accuracy: 0.6341
Epoch 4/120
14/14 [==============================] - 12s 859ms/step - loss: 0.8896 - accuracy: 0.6606 - val_loss: 0.8430 - val_accuracy: 0.6951
Epoch 5/120
14/14 [==============================] - 12s 847ms/step - loss: 0.8105 - accuracy: 0.7053 - val_loss: 0.7911 - val_accuracy: 0.7317
Epoch 6/120
14/14 [==============================] - 12s 839ms/step - loss: 0.7295 - accuracy: 0.7370 - val_loss: 0.7806 - val_accuracy: 0.6921
Epoch 7/120
14/14 [==============================] - 12s 850ms/step - loss: 0.6675 - accuracy: 0.7489 - val_loss: 0.6990 - val_accuracy: 0.

In [31]:
model.save('/content/model.h5')
files.download('/content/model.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [79]:
desired_batch_size = 32
filenames = test_data_gen.filenames
classes = {0:'cardboard', 1:'glass', 2:'metal', 3:'paper', 4:'plastic',5: 'trash'}
nb_samples = len(filenames)
predict_cnn = cnn.predict_generator(test_data_gen,steps = np.ceil(nb_samples/desired_batch_size))
predict = model.predict_generator(test_data_gen,steps = np.ceil(nb_samples/desired_batch_size))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  """


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  


In [74]:
classes

{0: 'cardboard', 1: 'glass', 2: 'metal', 3: 'paper', 4: 'plastic', 5: 'trash'}

In [77]:
# output=pd.DataFrame(filenames)
output= {}
for i in range(len(predict)):
  output[filenames[i]] = classes[np.argmax(predict[i])]

In [78]:
output

{'cardboard/cardboard105.jpg': 'metal',
 'cardboard/cardboard115.jpg': 'metal',
 'cardboard/cardboard121.jpg': 'cardboard',
 'cardboard/cardboard125.jpg': 'metal',
 'cardboard/cardboard128.jpg': 'cardboard',
 'cardboard/cardboard160.jpg': 'paper',
 'cardboard/cardboard162.jpg': 'paper',
 'cardboard/cardboard175.jpg': 'plastic',
 'cardboard/cardboard18.jpg': 'cardboard',
 'cardboard/cardboard184.jpg': 'plastic',
 'cardboard/cardboard19.jpg': 'cardboard',
 'cardboard/cardboard190.jpg': 'paper',
 'cardboard/cardboard2.jpg': 'trash',
 'cardboard/cardboard208.jpg': 'metal',
 'cardboard/cardboard210.jpg': 'paper',
 'cardboard/cardboard213.jpg': 'glass',
 'cardboard/cardboard214.jpg': 'glass',
 'cardboard/cardboard216.jpg': 'cardboard',
 'cardboard/cardboard218.jpg': 'glass',
 'cardboard/cardboard222.jpg': 'paper',
 'cardboard/cardboard231.jpg': 'cardboard',
 'cardboard/cardboard239.jpg': 'paper',
 'cardboard/cardboard248.jpg': 'paper',
 'cardboard/cardboard252.jpg': 'plastic',
 'cardboard/ca

In [80]:
# output=pd.DataFrame(filenames)
output_cnn= {}
for i in range(len(predict)):
  output_cnn[filenames[i]] = classes[np.argmax(predict_cnn[i])]

In [81]:
output_cnn

{'cardboard/cardboard105.jpg': 'plastic',
 'cardboard/cardboard115.jpg': 'plastic',
 'cardboard/cardboard121.jpg': 'paper',
 'cardboard/cardboard125.jpg': 'plastic',
 'cardboard/cardboard128.jpg': 'trash',
 'cardboard/cardboard160.jpg': 'glass',
 'cardboard/cardboard162.jpg': 'metal',
 'cardboard/cardboard175.jpg': 'cardboard',
 'cardboard/cardboard18.jpg': 'cardboard',
 'cardboard/cardboard184.jpg': 'trash',
 'cardboard/cardboard19.jpg': 'paper',
 'cardboard/cardboard190.jpg': 'cardboard',
 'cardboard/cardboard2.jpg': 'cardboard',
 'cardboard/cardboard208.jpg': 'glass',
 'cardboard/cardboard210.jpg': 'metal',
 'cardboard/cardboard213.jpg': 'cardboard',
 'cardboard/cardboard214.jpg': 'cardboard',
 'cardboard/cardboard216.jpg': 'glass',
 'cardboard/cardboard218.jpg': 'metal',
 'cardboard/cardboard222.jpg': 'cardboard',
 'cardboard/cardboard231.jpg': 'glass',
 'cardboard/cardboard239.jpg': 'metal',
 'cardboard/cardboard248.jpg': 'plastic',
 'cardboard/cardboard252.jpg': 'paper',
 'cardbo